<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">
 
# Decision Trees
 
_Authors: Joseph Nelson, David Yerrington, Matt Brems, Jeff Hale_

*Some adapted from Chapter 8 of ISLR*

---

## Learning Objectives

After this lesson you will be able to:

- Explain how a decision tree is created.
- Build a decision tree model in scikit-learn.
- Tune a decision tree model and explain how tuning effects the model.
- Interpret a tree diagram.
___


## Introduction

## Decision Trees

- Non-parametric models - what does that mean?
- Can be used for regression or classification. 

### Decision tree benefits:

- Explainable
- Interpretable
- Visualizable
- Basis for more sophisticated models.

---
## What will we get for dinner?

|$Y = $ Food|$X_1 = $ Weather|$X_2 = $ Day|
|:---------:|:--------------:|:----------:|
|   Indian  |      Rainy     |   Weekday  |
|   Sushi   |      Sunny     |   Weekday  |
|   Indian  |      Rainy     |   Weekend  |
|  Mexican  |      Sunny     |   Weekend  |
|   Indian  |      Rainy     |   Weekday  |
|  Mexican  |      Sunny     |   Weekend  |

<details><summary>It's a rainy day. Based on our past orders, what do you think we'll order?</summary>

- Indian food.
- In 100% of past cases where the weather is rainy, we've eaten Indian food!

|$Y = $ Food|$X_1 = $ Weather|$X_2 = $ Day|
|:---------:|:--------------:|:----------:|
|   Indian  |      Rainy     |   Weekday  |
|   Indian  |      Rainy     |   Weekend  |
|   Indian  |      Rainy     |   Weekday  |

</details>

<details><summary>It's a sunny day. Based on our past orders, what do you think we'll order?</summary>

- Either Sushi or Mexican food... but we can't say with certainty whether we'd eat sushi or Mexican food.
- Based on our past orders, we eat sushi on 1/3 of sunny days and we eat Mexican food on 2/3 of sunny days.
- If I **had** to make a guess here, I'd probably predict Mexican food, but we may want to use additional information to be certain.

|$Y = $ Food|$X_1 = $ Weather|$X_2 = $ Day|
|:---------:|:--------------:|:----------:|
|   Sushi   |      Sunny     |   Weekday  |
|  Mexican  |      Sunny     |   Weekend  |
|  Mexican  |      Sunny     |   Weekend  |

</details>

<details><summary>It's a sunny day that also happens to be a weekend. Based on our past orders, what do you think we'll order?</summary>

- Mexican food.
- In 100% of past cases where the weather is sunny and where it's a weekend, we've eaten Mexican food!

|$Y = $ Food|$X_1 = $ Weather|$X_2 = $ Day|
|:---------:|:--------------:|:----------:|
|  Mexican  |      Sunny     |   Weekend  |
|  Mexican  |      Sunny     |   Weekend  |

</details>

---
# Decision Trees: Overview

A decision tree for classification:
- takes a dataset consisting of $X$ and $Y$ data, 
- finds rules based on our $X$ data that splits our data into smaller datasets such that
- by the bottom of the tree, the values $Y$ in each "leaf node" are as "pure" as possible.

We frequently see decision trees represented by a graph.

<img src="images/order_food_dt.png" alt="order_food" width="750"/>

- (This image was created using [Draw.io](https://www.draw.io/).)

### Terminology
Decision trees look like upside down trees. 
- The top node is the "root node," through which all of our observations are passed.
- At each internal split, our dataset is partitioned.
- A "parent" node is split into two "child" nodes.
- At each of the "leaf nodes" (colored orange), we contain a subset of records that are as pure as possible.
    - In this food example, each leaf node is perfectly pure. Once we get to a leaf node, every observation in that leaf node has the exact same value of $Y$!
    - There are ways to quantify the idea of "purity" here so that we can let our computer do most of the tree-building (model-fitting) process... we'll come back to this later.


- [DecisionTreeClassifier Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)
- [DecisionTreeRegressor Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

---
## Purity in Decision Trees

When quantifying how "pure" a node is, we want to see what the distribution of $Y$ is in each node, then summarize this distribution with a number.

<img src="images/order_food_dt.png" alt="order_food" width="750"/>

---
#### For a continuous $y$ (i.e. using a decision tree to predict income), the default option is **mean squared error**.
- This is the `criterion = 'mse'` argument in `DecisionTreeRegressor`.
- When the decision tree is figuring out which split to make at a given node, it picks the split that maximizes the drop in MSE from the parent node to the child node.
    
    
---
#### For a discrete $y$, the default option is the **Gini impurity**, a measure of the homogeneity of the outcome class. 0 means totally homogeneous.
$$
\begin{eqnarray*}
\text{Gini impurity} &=& 1 - \sum_{i=1}^{classes} P(\text{class i})^2 \\
\text{Gini impurity (2 classes)} &=& 1 - P(\text{class 1})^2 - P(\text{class 2})^2 \\
\text{Gini impurity (3 classes)} &=& 1 - P(\text{class 1})^2 - P(\text{class 2})^2 - P(\text{class 3})^2 \\
\end{eqnarray*}
$$

Where P = probability

In [1]:
y = ['Indian', 'Sushi', 'Indian', 'Mexican', 'Indian', 'Mexican']

### Gini Practice

<details><summary>What is the Gini impurity of a node when every y is from the same class?</summary>
    
- Our Gini impurity is zero.

$$
\begin{eqnarray*}
\text{Gini impurity} &=& 1 - \sum_{i=1}^{classes} P(\text{class i})^2 \\
&=& 1 - P(\text{class 1})^2 \\
&=& 1 - 1^2 \\
&=& 1 - 1 \\
&=& 0
\end{eqnarray*}
$$
</details>

<details><summary>What is the Gini impurity of a node when we have two classes, each with two items?</summary>
    
- Our Gini impurity is 0.5.

$$
\begin{eqnarray*}
\text{Gini impurity} &=& 1 - \sum_{i=1}^{classes} P(\text{class i})^2 \\
&=& 1 - P(\text{class 1})^2 - P(\text{class 2})^2 \\
&=& 1 - \left(\frac{1}{2}\right)^2 - \left(\frac{1}{2}\right)^2 \\
&=& 1 - \frac{1}{4} - \frac{1}{4} \\
&=& \frac{1}{2}
\end{eqnarray*}
$$
</details>

<details><summary>What is the Gini impurity of a node when we have two classes, each with three items?</summary>
    
- Our Gini impurity is 0.5.

$$
\begin{eqnarray*}
\text{Gini impurity} &=& 1 - \sum_{i=1}^{classes} P(\text{class i})^2 \\
&=& 1 - P(\text{class 1})^2 - P(\text{class 2})^2 \\
&=& 1 - \left(\frac{1}{2}\right)^2 - \left(\frac{1}{2}\right)^2 \\
&=& 1 - \frac{1}{4} - \frac{1}{4} \\
&=& \frac{1}{2}
\end{eqnarray*}
$$
</details>

---
### How does a CLASSIFICATION decision tree decide which variable to split on?

1. At each node, consider the SUBSET of our DataFrame that exists at that node.
1. Iterate through EACH feature that could potentially split the data.
1. Calculate the Gini impurity for EACH possible split.
1. Select the feature split that DROPS the Gini impurity to the lowest level.
1. Repeat



A decision tree makes the best short-term decision by optimizing at each node individually. _This might mean that our tree isn't optimal in the long run!_ 

Decision tree fitting use a **greedy** algorithm because it makes locally optimal decisions. 

---
# Part 1: Regression Trees 🌲



## How Does a Computer Build a regression Tree? 

It has to do something different with a continuous y.


Recursive binary splitting.

1. Begin at the top of the tree.
2. For **every feature**, examine **every possible cutpoint**, and choose the feature and cutpoint so that the resulting tree has the **lowest possible mean squared error (MSE)**. Make that split.
3. Examine the two resulting regions. Once again, make a **single split** (in one of the regions) to minimize the MSE.
4. Keep repeating Step 3 until a **stopping criterion** is met:
    - Maximum tree depth (maximum number of splits required to arrive at a leaf).
    - Minimum number of observations in a leaf.

Remember this is a greedy algorithm (locally optimimal).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

### Read in vehicle data

In [ ]:
path = './data/vehicles_train.csv'
train = pd.read_csv(path)
train

In [ ]:
train.info()

#### Make X and y. Use year, miles, and doors to pedict price.

In [ ]:
X = train.drop(columns=['price', 'vtype'])
y = train['price']

#### Make train and test sets out of the train data


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)

#### Make a null baseline model and calculate the RMSE

---
## Build a regression tree
At every split, it will look at each feature for every potential cutpoint. It will choose the cutpoint that produces the lowest MSE.

#### Instantiate a DecisionTreeRegressor (with random_state=1).

### Create a Tree Diagram

Use the `plot_tree()` function. You may need to install a package in your conda environment.

---
### Every tree is just a graph in disguise! 🌳

A graph is made of nodes and edges (boxes and lines). A node is also called a vertex.

A tree is a graph with one node with no incoming edges (the root). Each other node has exactly one incoming edge (to its parent node).

#### Reading the graph:

- **rule:** Rule used to split that node (go left if true, go right if false).
- **samples:** Number of observations in that node before splitting.
- **mse:** MSE calculated by comparing the actual response values in that node against the mean response value in that node.
- **value:** Mean target value in that node (predicted value if a terminal node).

### What does a complete tree mean in terms of bias/variance?



---

<a id="#tuning-tree"></a>
## Tuning a Regression Tree

Let's try to reduce the MSE by tuning the **max_depth** parameter:

#### Try a few different values for `max_depth`.


### Find the most important features ⭐️
We don't have coefficients, but we have a feature importance metric.
#### _Gini importance_ of each feature: the (scaled, between 0 and 1) total reduction of error brought by that feature.

You can access the `feature_importances_` attribute through the `best_estimator_` attribute of the GridSearchCV object.

#### Which feature does the model think is most important?

### What happens when making predictions for new data?

#### Import **new** unseen data

#### Using the tree diagram above, what predictions will the model make for each observation?

#### Use the  fitted model to make predictions on final holdout testing data.

#### Calculate RMSE

---
# Part 2: Classification Trees

### Gini Index

- Gini index makes splits that **increase node purity**, even if that split does not change the classification error rate.
- Node purity is important because we're interested in the **class proportions** in each region. That's how we calculate the **predicted probability** of each class.
- scikit-learn's default splitting criteria for classification trees is gini.

---
## Building a Classification Tree in `scikit-learn`

We'll build a classification tree using the Titanic survival data set:

#### Read in the data.

In [ ]:
path = './data/titanic.csv'
titanic = pd.read_csv(path)

- **Survived:** 0=died, 1=survived (response variable)
- **Pclass:** 1=first class, 2=second class, 3=third class
- **Sex:** female, male
- **Age:** Numeric value
- **Embarked:** C or Q or S

In [ ]:
titanic.info()

#### Define X and y.

In [ ]:
feature_cols = ['Pclass', 'Age']

X = titanic[feature_cols]
y = titanic['Survived']

#### TTS 

Generally, for scikit-learn estimators, all our feature columns need to encoded numerically, but decision trees in scikit-learn will work with categorical data. But some scoring functions won't. ☹️

#### Fill missing values with KNN Imputation
Should standard scale first. We'll just go with this for now and add standard scaling later.

---
#### Fit a classification tree with max_depth=2 on all data.

#### Create a  visualization.

#### Compute the feature importances (the Gini index at each node).

#### Compute all the metrics and confusion matrix

#### How did our model do?

---
### KNN imputer for missing values

KNN imputing is based on distance, so we want to scale our data before imputing.

Make a pipeline with StandardScaler, KNNImputer, and a Decision Tree. GridSearch over two hyperparameters of your choosing for the Decision Tree. Max depth is a good one to help our model avoid overfitting. 🙂

#### Evaluate performance on the test set

Plot the confusion matrix and the best tree

--- 
### Disadvantages of decision trees:

- **Not that good (alone):** Their performance is (generally) not competitive with the best supervised learning methods.
- **Prone to overfitting:** They can easily overfit the training data (tuning is required).
- **Often have high variance**: Small variations in the data can result in a completely different tree.
- **Sensitive to class imbalance:** They don't tend to work well if the classes are highly unbalanced.
- **Need number of observations:** They don't tend to work well with very small data sets.

---
## Summary

Cool! You've seen the basics of decision trees! 🎉

Decision Trees are the foundation for a very powerful family models you'll learn about soon.

### Check for Understanding

- What are edges in a graph?
- What is the top node in a decision tree called?
- What are the bottom nodes called?
- What's the most common splitting criterion for classification trees?
- What's the most default splitting criterion for regression trees?
